In [3]:
from transformers import RobertaTokenizer, RobertaForCausalLM
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import os
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import lemmy
import stanza

In [4]:
pos_model = stanza.Pipeline("da", processors='tokenize,pos', use_gpu=True, cache_directory='./cache', tokenize_pretokenized=True, n_process=4)
lemmatizer = lemmy.load("da")

2023-03-26 23:37:13 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-03-26 23:37:14 INFO: Loading these models for language: da (Danish):
| Processor | Package |
-----------------------
| tokenize  | ddt     |
| pos       | ddt     |

2023-03-26 23:37:14 WARNING: GPU requested, but is not available!
2023-03-26 23:37:14 INFO: Using device: cpu
2023-03-26 23:37:14 INFO: Loading: tokenize
2023-03-26 23:37:14 INFO: Loading: pos
2023-03-26 23:37:14 INFO: Done loading processors!


In [6]:
def get_pos(sentence):
    doc = pos_model(sentence)
    return [word.upos for sentence in doc.sentences for word in sentence.words]

In [7]:
def convert_sentence(sentences):

    for x in tqdm(range(0, len(sentences))):
        cur_sentences = sentences[x]
        cur_sentence = " ".join(cur_sentences)
        words = cur_sentence.split()
        len_cur_sent = len(words)
        pos = np.array(get_pos(cur_sentence))
        verb_indices = np.where(pos == "VERB")[0]
        if len(pos) != len(words):
            continue
        for i in verb_indices:
            if i < 4:
                words_after = ["<pad>"]*(4-i) + words[0:i+5]
                words_before = ["<pad>"]*(4-i) + words[0:i] + [lemmatizer.lemmatize(pos[i], words[i])[0]] + words[i+1:i+5]
                value = 1
            elif i >= len_cur_sent-4:
                words_after = words[i-4:] + ["<pad>"]*(4-(len_cur_sent-i)+1)
                words_before = words[i-4:i] + [lemmatizer.lemmatize(pos[i], words[i])[0]] + words[i+1:] + ["<pad>"]*(4-(len_cur_sent-i)+1)
                value = 2
            else:
                words_after = words[i-4:i+5]
                words_before = words[i-4:i] + [lemmatizer.lemmatize(pos[i], words[i])[0]] + words[i+1:i+5]
                current_sentence.append([(" ".join(words_before)).lower(), (" ".join(words_after)).lower()])
                value = 3
            if len(words_before) != 9:
                print("Error:")
                print(words_before, value)
                print(i, len_cur_sent)
                continue

In [ ]:
test_set = pd.read_csv('../GrammatiktakDatasets/otherDatasets/nutids_r.csv', names=["wrong", "right"])
test_lines = list(test_set["wrong"])
